# Import Uplift calculation (loads needed data)

In [36]:
%load_ext autoreload
%autoreload

from Uplift import Uplift

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initialize and calculate uplift

In [40]:
uplift_test = Uplift(
    name="Uplift Test",
    variables={"g_220": [0], "md_agenatrep": [2, 3]},
    stations=[], #["Aarau", "Brig"],
    time_scale="Hour",
)
print(uplift_test)

Name: 'Uplift Test'
Stations: ['Aarau', 'Basel SBB', 'Bellinzona', 'Bern', 'Biel/Bienne', 'Brig', 'Chur', 'Fribourg', 'Genève Aéroport', 'Genève Cornavin', 'Lausanne', 'Lugano', 'Luzern', 'M2', 'Neuchatel', 'Olten', 'St. Gallen', 'Winterthur', 'Zug', 'Zürich Enge', 'Zürich Flughafen', 'Zürich Flughafen - Airside', 'Zürich Flughafen - Landside', 'Zürich HB', 'Zürich Hardbrücke', 'Zürich Oerlikon', 'Zürich Stadelhofen']
Timescale: 'Hour'
Selection: 
'Anzahl Autos im Haushalt' = 'Keines'
 AND 'Alter nach 4 Kategorien' = '46-60 Jahre' OR '61+ Jahre'
Results per Variable: 
{}
Total Result: 
Empty DataFrame
Columns: []
Index: []


In [41]:
%%time
uplift_test.calculate()

Wall time: 3.54 s


# Find best slots by uplift vs. population

In [32]:
def best_pop_uplift_slots(report: Uplift, top_n: int = 20):
    return (
        report.result.eval("pop_uplift_pers = pop_uplift * spr")
        .sort_values("pop_uplift_pers", ascending=False)
        .iloc[:top_n]["Station DayOfWeek Hour spr ax_ratio target_count pop_ratio pop_uplift pop_uplift_pers".split()]
        .sort_values(["Station", "DayOfWeek", "Hour"])
    )

In [39]:
best_pop_uplift_slots(uplift_test, 30)

,Station,DayOfWeek,Hour,spr,ax_ratio,target_count,pop_ratio,pop_uplift,pop_uplift_pers
14,Aarau,Monday,14,665.924705,0.155059,103.257421,0.079101,0.075958,50.582168
15,Aarau,Monday,15,746.678547,0.188252,140.563812,0.079101,0.109151,81.500855
16,Aarau,Monday,16,1040.020026,0.129352,134.528701,0.079101,0.050251,52.262165
17,Aarau,Monday,17,1424.549830,0.181313,258.288926,0.079101,0.102212,145.605731
19,Aarau,Monday,19,763.068955,0.166465,127.024322,0.079101,0.087364,66.664869
33,Aarau,Tuesday,09,593.952093,0.177636,105.507383,0.079101,0.098535,58.525229
34,Aarau,Tuesday,10,516.882256,0.199170,102.947245,0.079101,0.120069,62.061386
38,Aarau,Tuesday,14,637.816990,0.146993,93.754621,0.079101,0.067892,43.302713
39,Aarau,Tuesday,15,729.916795,0.216439,157.982662,0.079101,0.137338,100.245576
41,Aarau,Tuesday,17,1458.920761,0.160075,233.536843,0.079101,0.080974,118.134876
